In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df_chicago_weather = pd.read_csv('../dataset/weather_hourly_chicago.csv')
df_chicago_weather.head()

In [ ]:
len(df_chicago_weather)

In [ ]:
df_chicago_weather.info()

In [ ]:
df_chicago_weather[df_chicago_weather.duplicated(keep=False)]

In [ ]:
df_chicago_weather = df_chicago_weather.drop_duplicates(keep='last')

In [ ]:
df_chicago_weather[df_chicago_weather.duplicated(keep=False)]

In [ ]:
df_chicago_weather.loc[df_chicago_weather['max_temp'] < df_chicago_weather['min_temp']]

In [ ]:
df_chicago_weather.isnull().sum()

In [ ]:
df_chicago_weather = df_chicago_weather.dropna(axis=0, inplace=False)

In [ ]:
df_chicago_weather.isnull().sum()

In [ ]:
len(df_chicago_weather)